# LSTM을 이용한 금 선물 가격 예측
- svm => "반복횟수가 모자라요!"
- 24GB[(X/32) * 1000]
- 시퀸스 길이: 30일

In [1]:
import os

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import torch
import torch.nn as nn
from torch.utils.data import DataLoader, TensorDataset

# 한글 폰트 설정
plt.rcParams['font.family'] = ['Malgun Gothic']
plt.rcParams['axes.unicode_minus'] = False

# GPU 사용 가능 여부 확인
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda


### 데이터 가져오기

In [2]:
def load_data(file_path):
    df = pd.read_csv(file_path, index_col="Date", parse_dates=True)
    return df

def split_data(df, train_ratio=0.8, val_ratio=0.1):
    train_size = int(len(df) * train_ratio)
    val_size = int(len(df) * val_ratio)
    train_df = df.iloc[:train_size]
    val_df = df.iloc[train_size : train_size + val_size]
    test_df = df.iloc[train_size + val_size :]
    return train_df, val_df, test_df

In [6]:
def create_scaler(train_df):
    scaler = {}
    for col in train_df.columns:
        scaler[col] = {"min": train_df[col].min(), "max": train_df[col].max()}
    return scaler

In [7]:
def scale_data(data, scaler):
    scaled_data = data.copy()
    for col in data.columns:
        min_val = scaler[col]["min"]
        max_val = scaler[col]["max"]
        scaled_data[col] = (data[col] - min_val) / (max_val - min_val)
    return scaled_data

In [8]:
def create_multivariate_sequences(data, target_col, seq_length):
    xs, ys = [], []
    data_np = data.values
    target_idx = data.columns.get_loc(target_col)

    for i in range(len(data_np) - seq_length):
        x = data_np[i : i + seq_length]
        y = data_np[i + seq_length, target_idx]
        xs.append(x)
        ys.append(y)
    return np.array(xs), np.array(ys).reshape(-1, 1)


In [14]:
def create_data_loaders(X_train, y_train, X_val, y_val, batch_size):
    X_train = torch.from_numpy(X_train).float()
    y_train = torch.from_numpy(y_train).float()
    X_val = torch.from_numpy(X_val).float()
    y_val = torch.from_numpy(y_val).float()

    # TODO1
    train_dataset = TensorDataset(X_train, y_train)
    train_loader = DataLoader(train_dataset, batch_size, shuffle=True)
    
    val_dataset = TensorDataset(X_val, y_val)
    val_loader = DataLoader(val_dataset, batch_size, shuffle=False)
    
    return train_loader, val_loader

In [19]:
class SequenceModel(nn.Module):
    # TODO2
    def __init__(self, input_size=3, hidden_size=50, num_layers=2, output_size=1):
        super(SequenceModel, self).__init__()
        self.lstm = nn.LSTM(input_size=input_size, 
                            hidden_size=hidden_size, 
                            num_layers=num_layers, 
                            batch_first=True, # (batch, seq, feature) 반드시 이 순서로 되게 True로 설정한다
                            dropout=0.2)
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x):
        out, _ = self.lstm(x) # output, (h_n, c_n) # (N, L, D * H_{out})
        out = self.fc(out[:, -1, :]) # 
        # batch_size, seq_len, hidden_size
        return out

In [ ]:
class EarlyStopping:
    def __init__(self, patience=10, verbose=False, delta=0, path="checkpoint.pt"):
        self.patience, self.verbose, self.delta, self.path = (
            patience,
            verbose,
            delta,
            path,
        )
        self.counter, self.best_score, self.early_stop, self.val_loss_min = (
            0,
            None,
            False,
            np.inf,
        )

    def __call__(self, val_loss, model):
        score = -val_loss
        if self.best_score is None:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
        elif score < self.best_score + self.delta:
            self.counter += 1
            if self.verbose:
                print(f"EarlyStopping counter: {self.counter} out of {self.patience}")
            if self.counter >= self.patience:
                self.early_stop = True
        else:
            self.best_score = score
            self.save_checkpoint(val_loss, model)
            self.counter = 0

    def save_checkpoint(self, val_loss, model):
        if self.verbose:
            print(
                f"Validation loss decreased ({self.val_loss_min:.6f} --> {val_loss:.6f}). Saving model..."
            )
        torch.save(model.state_dict(), self.path)
        self.val_loss_min = val_loss

In [22]:
def train_model(
    model, train_loader, val_loader, device, num_epochs, learning_rate, patience=10
):
  # TODO3
  # 손실함수
  criterion = nn.MSELoss()
  # 최적화함수
  optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
  # ES
  early_stopping = EarlyStopping(patience=patience, verbos=True) # verbos는 학습할때만 True로 실제 서비스시엔 False

  # 반복문(num_epochs)
  for each in range(num_epochs):
    model.train()
    for batch_X, batch_y in train_loader:
      batch_X, batch_y = batch_X.to(device), batch_y.to(device)
      output = model(batch_X)
      loss = criterion(optimizer, batch_y)

      optimizer.zero_grad()
      loss.backward()
      optimizer.step()
    model.eval() # 모델을 얼린다
    val_losses = []
    with torch.no_grad():
      for batch_X, batch_y in val_loader:
        batch_X, batch_y = batch_X.to(device), batch_y.to(device)
        output = model(batch_X)
        val_losses.append(criterion(output, batch_y).item())

    val_loss = np.mean(val_losses)
    print(f"Epoch [{each+1} / {num_epochs}]")

    early_stop = early_stopping(val_loss, model)

    if early_stop.ear:
      print("Early Stop")
      break # 매우 중요!!!!!

  # 학습모드 => train_loader
  # 평가모드 => val_loader

  # 모델을 반환
  model.load_state_dict(torch.load("checkpoint.pt"))
  return model

SyntaxError: invalid syntax (1538069226.py, line 15)

In [ ]:
def evaluate_and_visualize(
    model, X_test, y_test, test_df, scaler, target_column, seq_length, device
):
    model.eval()
    with torch.no_grad():
        test_outputs = model(X_test.to(device)).cpu().numpy()

    predicted_prices = inverse_scale_data(test_outputs, scaler, target_column)
    actual_prices = inverse_scale_data(y_test.numpy(), scaler, target_column)

    plt.figure(figsize=(15, 7))
    plt.title(f"Futures Gold Price Prediction (LSTM)")
    plt.plot(
        test_df.index[seq_length:], actual_prices, label="Actual Price", color="blue"
    )
    plt.plot(
        test_df.index[seq_length:],
        predicted_prices,
        label="Predicted Price",
        color="red",
        linestyle="--",
    )
    plt.xlabel("Date")
    plt.ylabel("Price")
    plt.legend()
    plt.grid(True)
    plt.show()

    return predicted_prices, actual_prices

In [ ]:
def inverse_scale_data(data, scaler, target_col):
    min_val = scaler[target_col]["min"]
    max_val = scaler[target_col]["max"]
    return data * (max_val - min_val) + min_val


In [ ]:
file_path = "data/stock_bond_futures_data.csv"
target_col = "Futures_Gold"
seq_length = 30 # LSTM에서는 하이퍼 파라미터다. 아주 중요하다! 주식의 1주일 - 5일, 선물의 1주일 - 7일
batch_size = 32 # 보통 8부터 시작해서 리소스 사용량 간보고 수치 잡는다.
num_epochs = 100
learning_rate = 0.001
patience = 10

In [4]:
df = load_data(file_path)
train_df, val_df, test_df = split_data(df)

In [10]:
scaler = create_scaler(train_df)
trained_scaled = scale_data(train_df, scaler)
val_scaled = scale_data(val_df, scaler)
test_scaled = scale_data(test_df, scaler)

In [12]:
X_train, y_train = create_multivariate_sequences(trained_scaled, target_col, seq_length)
X_val, y_val = create_multivariate_sequences(val_scaled, target_col, seq_length)
X_test, y_test = create_multivariate_sequences(test_scaled, target_col, seq_length)

In [ ]:
train_loader, val_loader = create_data_loaders(X_train, y_train, X_val, y_val, batch_size)
X_test_tensor = torch.from_numpy(X_test).float()
y_test_tensor = torch.from_numpy(y_test).float()

#### ===== 데이터 가져오기 끝

In [ ]:
model = SequenceModel(input_size=X_train.shape[2]).to(device)
train_model = train_model(model, train_loader, val_loader, device, num_epochs, learning_rate, patience)

In [ ]:
print(X_train.shape, X_val.shape, X_test.shape)
# (4609, 30, 3)
# 4609 => 
# 30 => seq_length
# 3 => feature 수

(4609, 30, 3) (549, 30, 3) (551, 30, 3)


In [17]:
y_train.shape

(4609, 1)

In [ ]:
# run
# TODO1

In [ ]:
# out[50,11,32]
# out[:,-1,:]
# [50,11,32]

# out[:,-1,:] (32,30,50) -1은 맨 마지막
# (32,30,50) => (32, 50)
# - 배치 (4609 데이터를 사용해서)
# - 시퀸스 (30개의 데이터를 학습)
# - 은닉층 (50개의 메모리를 활용)
# (32,30,50) => self.fc((32, 50))
# => 30일까지 학습하고 1개를 예측

# 1. LSTM 할 때, 반드시 시점 이후 뭘 보고 싶은건지 꼭 먼저 정해야 함
# 